In [1]:
import tensorflow as tf

In [3]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10

from matplotlib import pyplot as plt

In [4]:
LABELS = [ 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck' ]
TRAIN_SUBSET = 10000 
TEST_SUBSET = 1000

In [11]:
def get_new_model(input_shape):
    model = Sequential([
         Conv2D(filters=16, input_shape=input_shape, kernel_size=(3, 3), activation='relu', name='conv_1'),
         Conv2D(filters=8, input_shape=input_shape, kernel_size=(3, 3), activation='relu', name='conv_2'),
         MaxPool2D(pool_size=(4, 4), name='pool_1'),
         Flatten(name='flatten'), 
         Dense(units=32, activation='relu', name='dense_1'),
         Dense(units=10, activation='softmax', name='dense_2')
      ])
    loss = SparseCategoricalCrossentropy()
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    return model

In [12]:
def get_test_accuracy(model, X_test, y_test):
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print('accuracy: {test_acc}'.format(test_acc=test_acc))

In [13]:
# X_train.shape = (32, 32, 3)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train / 255.
X_test = X_test / 255.

X_train = X_train[:TRAIN_SUBSET]
y_train = y_train[:TRAIN_SUBSET]

X_test = X_test[:TEST_SUBSET]
y_test = y_test[:TEST_SUBSET]

model = get_new_model(X_train[0].shape)

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 28, 28, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 7, 7, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 392)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                12576     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 14,514
Trainable params: 14,514
Non-trainable params: 0
__________________________________________________

In [10]:
X_train[0].shape

(32, 32, 3)

In [15]:
print('Train set = ',TRAIN_SUBSET)

('Train set = ', 10000)


In [16]:
checkpoint = ModelCheckpoint('checkpoints/cifar_5000_{epoch:02d}_{val_loss}',
                             save_weights_only=True, save_freq=5000, verbose=1)

In [18]:
print('X_train = ', X_train.shape)
print('X_test = ', X_test.shape)

('X_train = ', (10000, 32, 32, 3))
('X_test = ', (1000, 32, 32, 3))


In [22]:
bs = 100
model.fit(x=X_train, y=y_train, 
          epochs=3, batch_size=bs,
          steps_per_epoch=X_train.shape[0] / bs,
          validation_data=(X_test, y_test),
          #callbacks=[ checkpoint ], 
          verbose=1)

Train on 10000 samples, validate on 1000 samples
Epoch 1/3
10000/10000 [==============================] - 11s 1ms/sample - loss: 1.3763 - accuracy: 0.5112 - val_loss: 1.3996 - val_accuracy: 0.5020
Epoch 2/3
10000/10000 [==============================] - 11s 1ms/sample - loss: 1.3120 - accuracy: 0.5300 - val_loss: 1.3822 - val_accuracy: 0.5050
Epoch 3/3
10000/10000 [==============================] - 12s 1ms/sample - loss: 1.2623 - accuracy: 0.5502 - val_loss: 1.3324 - val_accuracy: 0.5350
